In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from AI_Trading.src.augmentation import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from tqdm import tqdm
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
create_dir()

In [ ]:
# training setting
model_name = 'DDPG'
exp = 'DDPG_OHLC(normalized)_alpha0_3y'

# create training log folder
save_path = os.path.join(config.LOG_PATH, exp)
if not os.path.isdir(save_path):
     try:
          os.mkdir(save_path)
     except Exception:
          print(f'no folder {save_path}')
          pass

In [ ]:
# train
for i in tqdm(range(len(config.TRAIN_START_DATE))):
    # create training log
    training_log_path = f'{config.LOG_PATH}/{exp}/training_log_{model_name}_{i}.csv'
    training_weight_path = f'{config.LOG_PATH}/{exp}/training_weight_{model_name}_{i}.csv'
    training_share_path = f'{config.LOG_PATH}/{exp}/training_share_{model_name}_{i}.csv'
    if os.path.exists(training_log_path):
        os.remove(training_log_path)

    if os.path.exists(training_weight_path):
        os.remove(training_weight_path)

    if os.path.exists(training_share_path):
        os.remove(training_share_path)

    train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i])
    env_kwargs = {
        "training_log_path": training_log_path,
        "training_weight_path": training_weight_path,
        "training_share_path": training_share_path,
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
        "add_cash": False,
        "alpha": 0,
        "add_window": 0
    }
    print(env_kwargs)
    print('config.add_window:', config.ADD_WINDOW)
    # env_train = maxPortfolioReturnEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = maxPortfolioReturnEnv(df = trade, is_test_set=True, **env_kwargs)

    env_train = riskSensitiveEnv(df = train, is_test_set=False, **env_kwargs)
    env_trade = riskSensitiveEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = trendConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = trendConcernEnv(df = trade, is_test_set=True, **env_kwargs)
    
    # env_train = varianceEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = varianceEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = contrarianEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = contrarianEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = downsideRiskEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = downsideRiskEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = macdConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = macdConcernEnv(df = trade, is_test_set=True, **env_kwargs)

    model_index = i
    # episode = 80
    # total_timesteps = len(train)/len(train.tic.unique())*episode
    total_timesteps = 10**5
    trainPortfolioAllocation(exp, env_train, model_name, model_index,total_timesteps=total_timesteps)

In [ ]:
#evaluate setting
model_name = 'DDPG'
exp = 'DDPG_OHLC(normalized)_alpha0_3y'
save_path = os.path.join(config.RESULTS_DIR, exp)
training_log_path = f'{config.LOG_PATH}/{exp}/training_log_{exp}_0.csv'
training_weight_path = f'{config.LOG_PATH}/{exp}/training_weight_{exp}_0.csv'
training_share_path = f'{config.LOG_PATH}/{exp}/training_share_{exp}_0.csv'
if not os.path.isdir(save_path):
    try:
        os.mkdir(save_path)
    except Exception:
        print(f'no folder {save_path}')
        pass


In [ ]:
df_annualReturn_allYear = pd.DataFrame()
df_mdd_allYear = pd.DataFrame()
df_sharpe_allYear = pd.DataFrame()
df_sortino_allYear = pd.DataFrame()
df_calmar_allYear = pd.DataFrame()
df_annualReturn_rank_allYear = pd.DataFrame()
df_mdd_rank_allYear = pd.DataFrame()
df_sharpe_rank_allYear = pd.DataFrame()
df_sortino_rank_allYear = pd.DataFrame()
df_calmar_rank_allYear = pd.DataFrame()

# evaluate
for i in tqdm(range(len(config.TEST_START_DATE))):
    train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i])
    env_kwargs = {
        "training_log_path": training_log_path,
        "training_weight_path": training_weight_path,
        "training_share_path": training_share_path,
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
        "add_cash": False,
        "alpha": 0,
        "add_window": 0
    }
    print(env_kwargs)
    print('config.add_window:', config.ADD_WINDOW)
    # env_train = maxPortfolioReturnEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = maxPortfolioReturnEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = trendConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = trendConcernEnv(df = trade, is_test_set=True, **env_kwargs)

    env_train = riskSensitiveEnv(df = train, is_test_set=False, **env_kwargs)
    env_trade = riskSensitiveEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = varianceEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = varianceEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = contrarianEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = contrarianEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = downsideRiskEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = downsideRiskEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = macdConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = macdConcernEnv(df = trade, is_test_set=True, **env_kwargs)
    model_index = i
    
    # load model
    model_zip_path = f'{config.TRAINED_MODEL_PATH}/{exp}/{model_name}_{str(model_index)}.zip'
    if model_name == 'A2C':
        model = A2C.load(model_zip_path,seed=0)
    elif model_name == 'PPO':
        model = PPO.load(model_zip_path,seed=0)
    elif model_name == 'DDPG':
        model = DDPG.load(model_zip_path,seed=0)
    elif model_name == 'TD3':
        model = TD3.load(model_zip_path)
    elif model_name == 'SAC':
        model = SAC.load(model_zip_path)

    # test
    df_daily_return, df_actions = test_portfolioAllocation(model, env_trade)
    df_actions.to_csv(f'{save_path}/df_action_{model_name}_{model_index}.csv')

    # evaluate
    print("==============DRL Strategy Stats===========")
    df_return, df_portfolio_value = computeReturns(df_actions,trade, transCostRate=0.001)
    returns, DRL_stats = getStats(df_return)
    print(DRL_stats)

    print("==============Equal Weight Strategy Stats===========")
    equalWeight_actions = getEqualWeightActions(trade)
    df_equalWeight_return, df_portfolio_value = computeReturns(equalWeight_actions,trade, transCostRate= 0.001)
    equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)
    print(equalWeight_stats)

    print("==============Min-Variance Strategy Stats===========")
    # minVariance_actions = getMinVarianceActions(trade)
    # df_minVariance_return, df_minVariance_portfolio_value = computeReturns(minVariance_actions,trade, transCostRate=0.001)
    # minVariance_returns, minVariance_stats = getStats(df_minVariance_return)
    # print(minVariance_stats)

    print("==============All In Stock Strategy Stats===========")
    all_stock_actions= getTicActions(trade, 'VTI')
    df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade, transCostRate= 0.001)
    stock_returns, stock_stats = getStats(df_stock_return)
    print(stock_stats)

    print("==============All In Debt Strategy Stats===========")
    all_debt_actions= getTicActions(trade, 'TLT')
    df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade, transCostRate= 0.001)
    debt_returns, debt_stats = getStats(df_debt_return)
    print(debt_stats)

    print("==============All In Reit Strategy Stats===========")
    all_reit_actions= getTicActions(trade, 'VNQ')
    df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade, transCostRate= 0.001)
    reit_returns, reit_stats = getStats(df_reit_return)
    print(reit_stats)
    
    df_stats = pd.concat({'DRL': DRL_stats,
                          'equalWeight': equalWeight_stats,
                        #   'minVariance':minVariance_stats,
                          'stock': stock_stats,
                          'debt': debt_stats,
                          'reit': reit_stats},axis=1)

    df_annualReturn_allYear = stats_allYear(i, 'Annual return', df_stats, df_annualReturn_allYear)
    df_mdd_allYear = stats_allYear(i, 'Max drawdown', df_stats, df_mdd_allYear)
    df_sharpe_allYear = stats_allYear(i, 'Sharpe ratio', df_stats, df_sharpe_allYear)
    df_sortino_allYear = stats_allYear(i, 'Sortino ratio', df_stats, df_sortino_allYear)
    df_calmar_allYear = stats_allYear(i, 'Calmar ratio', df_stats, df_calmar_allYear)
    
    df_annualReturn_rank_allYear = rankCaculate(i, 'Annual return', df_stats, df_annualReturn_rank_allYear)
    df_mdd_rank_allYear = rankCaculate(i, 'Max drawdown', df_stats, df_mdd_rank_allYear)
    df_sharpe_rank_allYear = rankCaculate(i, 'Sharpe ratio', df_stats, df_sharpe_rank_allYear)
    df_sortino_rank_allYear = rankCaculate(i, 'Sortino ratio', df_stats, df_sortino_rank_allYear)
    df_calmar_rank_allYear = rankCaculate(i, 'Calmar ratio', df_stats, df_calmar_rank_allYear)

In [ ]:
average_allYear(df_annualReturn_allYear)

In [ ]:
average_allYear(df_mdd_allYear)

In [ ]:
average_allYear(df_calmar_allYear)

In [ ]:
average_allYear(df_sharpe_allYear)

In [ ]:
average_allYear(df_sortino_allYear)

In [ ]:
df_annualReturn_rank_allYear_avg = average_allYear(df_annualReturn_rank_allYear)
df_annualReturn_rank_allYear_avg

In [ ]:
df_mdd_rank_allYear_avg  = average_allYear(df_mdd_rank_allYear)
df_mdd_rank_allYear_avg 

In [ ]:
df_sharpe_rank_allYear_avg = average_allYear(df_sharpe_rank_allYear)
df_sharpe_rank_allYear_avg 

In [ ]:
df_sortino_rank_allYear_avg = average_allYear(df_sortino_rank_allYear)
df_sortino_rank_allYear_avg

In [ ]:
df_calmar_rank_allYear_avg = average_allYear(df_calmar_rank_allYear)
df_calmar_rank_allYear_avg 